Recognizing Shapes in Objects with OpenCV
=========================================


Import libraries.

In [15]:
import cv2
import numpy as np
import os # Contains os.path python method/function

Image filenames.

In [16]:
#Tuple data structure is used to store file names

files = (   
    'breakfast.jpg',
    'landscape-house.jpg',
     'dinner.jpg',
    'building.jpg'
)

# Try the format image_file = r'..\..\..\images\xxx.jpg' if any problems with reading an image.

# "Set x[0 to 2] in files[x] in order read the desired image"
image_file = os.path.join("images",files[2])


Define a function for viewing images.

In [17]:
def imageviewer(f):
    cv2.imshow('view',f)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    

Read an image from file.

In [18]:
# Check if image can be read.
b = cv2.haveImageReader(image_file)
print(b)

# Read and show image
i=cv2.imread(image_file)
imageviewer(i)

True


Inspect image contents

In [19]:
print(i.shape)
print(i[0,0,:])

(427, 640, 3)
[43 47 52]


Gray-scale

In [20]:
# Need to convert image to gray scale as most tools wouldn't work well with RGB images
gray_img = cv2.cvtColor(i,cv2.COLOR_BGR2GRAY)
imageviewer(gray_img)
print(gray_img[0,0])
print(gray_img.shape)

48
(427, 640)


X gradient

In [21]:
# Differentitate with Sobel function in OpenCV.This will be useful for detection of edges.
# Look for brightness increases in the x-direction with following code.
x_gradient = cv2.Sobel(gray_img,cv2.CV_64F,1,0)

# Need to get postive values
x_gradient_abs = np.absolute(x_gradient)

# Divide by maximum value/brightness to normalize to between 0 and 1.
imageviewer(x_gradient_abs/np.max(x_gradient_abs))

Y gradient

In [22]:
# Look for brightness increases in the y-direction with following code.
y_gradient = cv2.Sobel(gray_img,cv2.CV_64F,0,1)

# Need to get postive values
y_gradient_abs = np.absolute(y_gradient)

# Divide by maximum value/brightness to normalize to between 0 and 1.
imageviewer(y_gradient_abs/np.max(y_gradient_abs))

Magnitude of gradient vector

In [23]:
# Sum x and y gradients
sum = np.sqrt(x_gradient**2+y_gradient**2)
imageviewer(sum/(np.max(sum)))

Canny edge detection

In [24]:
edges = cv2.Canny(gray_img,50,100) #Adjust integer parameters to either reduce or increase number of edges shown. Try changing the high/low thresholds 100/50 to 150/100
                  
imageviewer(edges)

Hough transform for lines

In [25]:
# Detecting lines using Hough transforms where x and y coordinates are replaced by parametric rho and theta coordinates because you can get infinite gradients using the xy coordinate system.
# rho => y pixels and theta => x pixels

lines = cv2.HoughLinesP(edges,rho=1,theta=1.*np.pi/180.0,threshold=20,minLineLength=25,maxLineGap=5)
i_lines = i.copy()

for l in lines:
    x1,y1,x2,y2 = l[0]
    cv2.line(i_lines,(x1,y1),(x2,y2),(0,0,255),thickness=3)
    
imageviewer(i_lines)


Hough transform for circles

In [26]:
# References: 1) https://theailearner.com/tag/cv2-houghcircles/
#             2) https://www.delftstack.com/howto/python/opencv-draw-circle/
#             3) https://pyimagesearch.com/2014/07/21/detecting-circles-images-using-opencv-hough-circles/

# Apply Hough transform to greyscale image
circles = cv2.HoughCircles(gray_img,cv2.HOUGH_GRADIENT,dp=2,minDist=35,param1=150,param2=40,minRadius=15,maxRadius=25)

# convert the (x, y) coordinates and radius of the circles to integers
circles = np.uint16(np.around(circles))


# Draw the circles
for x in circles[0,:]:
    # draw the outer circle
    cv2.circle(i,(x[0],x[1]),x[2],(0,0,255),2)
    # draw the center of the circle
    cv2.circle(i,(x[0],x[1]),2,(0,0,255),3)


imageviewer(i)

'''if circles is not None:
    # convert the (x, y) coordinates and radius of the circles to integers
    circles = np.round(circles[0, :]).astype("int")


detected_circles = np.uint16(np.around(circles))
detected_circles = circles.copy()

for x, y, r in circles[0,:]:
        cv2.circle(detected_circles, x, y,r, (0, 0, 255), 3)
        #cv.circle(detected_circles, (x, y), 2, (255, 0, 0), 3)


imageviewer(circles)
'''

'if circles is not None:\n    # convert the (x, y) coordinates and radius of the circles to integers\n    circles = np.round(circles[0, :]).astype("int")\n\n\ndetected_circles = np.uint16(np.around(circles))\ndetected_circles = circles.copy()\n\nfor x, y, r in circles[0,:]:\n        cv2.circle(detected_circles, x, y,r, (0, 0, 255), 3)\n        #cv.circle(detected_circles, (x, y), 2, (255, 0, 0), 3)\n\n\nimageviewer(circles)\n'

Blur the image first

In [27]:
#Blur image to reduce nosie in order to improve circle detection
#blurry_img = cv2.GaussianBlur(gray_img,ksize=(13,13),sigmaX=0,)
blurry_img = cv2.blur(gray_img,(9,9))

#View without circles: 
imageviewer(blurry_img)



Circle detection on blurred image

In [28]:
# Circle detection code reused from above
circles = cv2.HoughCircles(blurry_img,cv2.HOUGH_GRADIENT,dp=2,minDist=35,param1=150,param2=40,minRadius=15,maxRadius=25)


# convert the (x, y) coordinates and radius of the circles to integers
circles = np.uint16(np.around(circles))


# Draw the circles
for x in circles[0,:]:
    # draw the outer circle
    cv2.circle(i,(x[0],x[1]),x[2],(0,0,225),2)
    #draw the center of the circle
    cv2.circle(i,(x[0],x[1]),2,(0,0,255),3)
    

imageviewer(i)
